[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/vrra/FGAN-Build-a-thon-2022/blob/main/Notebooks2022/build_a_thon_graph_v1.ipynb)

## Build-a-thon 2022, Hoist the colors?

In [ ]:
!rm -rf /tmp/FGAN-Build-a-thon-2022/
!git clone https://github.com/vrra/FGAN-Build-a-thon-2022.git /tmp/FGAN-Build-a-thon-2022


In [ ]:
from IPython.display import Image
Image(filename='/tmp/FGAN-Build-a-thon-2022/Banner/1920x1080-FG_AN-buildathon_Sep02-01.png', width=500) 


In [ ]:
!rm -rf /tmp/FGAN-Build-a-thon-2022/

## Introduction

Created: 14 Apr 2022.   
Author: Vishnu Ram OV.   
vishnu.n@ieee.org.   
Licence: available for all purposes under the sun  
but with acknowledgement and citation to "ITU FG AN Build-a-thon 2022"


## References 
[1] Neo4j Docs, "Creating an instance", https://neo4j.com/docs/aura/auradb/getting-started/create-database/   
[2] ITU-T Focus Group Autonomous Networks, "Use cases for Autonomous Networks", https://extranet.itu.int/sites/itu-t/focusgroups/an/output/FGAN-O-013-R1.docx  
[3] Build-a-thon github, https://github.com/vrra/FGAN-Build-a-thon-2022

## Prerequisite: 
create a free account at https://console.neo4j.io  
Follow steps at https://extranet.itu.int/sites/itu-t/focusgroups/an/input/FGAN-I-253.pdf  


## General cautions
Property names have strict conventions and rules in neo4j (apparently)

### For Colab users:

If you are using colab, add your credentials here and uncomment these  
and skip to step-1 below.

In [ ]:
#uri = "your uri from console.neo4j.io"  
#user = "your username from console.neo4j.io"  
#password = "your passwd from console.neo4j.io"  

### For non-colab users:
Create a local file .env with   
neo4j_uri="your uri from console.neo4j.io"  
neo4j_user="your username from console.neo4j.io"  
neo4j_password="your passwd from console.neo4j.io"  

#### Read the neo4j URI, username, passwd


In [6]:
import os
%pip install python-dotenv
from dotenv import load_dotenv
load_dotenv()

Note: you may need to restart the kernel to use updated packages.


True

In [7]:

#Vishnu: 1 Jun 2022: use the details from https://console.neo4j.io
#especially use your own DB instance and corresponding passwd.
#if you dont have a .env file, replace here with your credentials.

uri = os.getenv('neo4j_uri')
user = os.getenv('neo4j_user')
password = os.getenv('neo4j_password')

## step-1: install neo4j

In [ ]:
%pip install neo4j

## step-2: imports and init

In [4]:
from neo4j import GraphDatabase
import logging
from neo4j.exceptions import ServiceUnavailable

## step-3: define helper class

In [3]:


class App:

    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        # Don't forget to close the driver connection when you are finished with it
        self.driver.close()

    #Vishnu: 1 Aug 2022: uses labels instead of properties
    #Vishnu: 1 Aug 2022: Invariant: only 1 actor with 1 name.
    #        (Note that rel still uses properties and not labels)
    def create_node_with_usecase_label(self, actor_name, usecase_id):
        with self.driver.session() as session:
            node_already_exists = session.read_transaction(
                self._find_and_return_existing_node_label, actor_name)
            if not node_already_exists:  
                #node itself doesnt exist (not to mention the use case label), Lets add.
                result = session.write_transaction(
                    self._create_and_return_node_label, actor_name, usecase_id)
                for row in result:
                    print("Created node: {n1}".format(n1=row['n1name']))
            else:
                for row in node_already_exists:
                    print("Node already exists: {name} in {usecase}".format(name=row['n1name'], 
                                                                 usecase=row['n1usecase']))
                    if not (usecase_id in row['n1usecase']):
                      print("adding label "
                                +usecase_id+ 
                                " in addition to {usecase}".format(usecase=row['n1usecase']))
                      result = session.write_transaction(
                                self._add_usecase_and_return_existing_node_label, actor_name, usecase_id)
                    else:
                      print("ignoring ...")
                    #row has only 1 entry due to invariant. So break here. and exit f()
                    break
    
    #Vishnu: 1 Aug 2022: created this f() to query nodes based on a name
    #                    and return with usecase labels
    @staticmethod
    def _find_and_return_existing_node_label(tx, actor_name):
        query = (
            "MATCH (n1) "
            "WHERE (n1.name ='" + actor_name + "') "
            "RETURN n1.name as n1name, labels(n1) as n1usecase"
        )
        result = tx.run(query)
        try:
            return [{"n1name": row["n1name"],"n1usecase": row["n1usecase"]} 
                    for row in result]
        # Capture any errors along with the query and data for traceability
        except ServiceUnavailable as exception:
            logging.error("{query} raised an error: \n {exception}".format(
                query=query, exception=exception))
            raise

    #Vishnu: 1 Aug 2022: created this f() to create nodes with usecase label
    @staticmethod
    def _create_and_return_node_label(tx, actor_name, usecase_id):
            query = (
            "CREATE (n1 :" + usecase_id + " { name: '" + actor_name + "'}) "
            "RETURN n1"
            )
            result = tx.run(query, actor_name=actor_name)
            try:
                return [{"n1name": row["n1"]["name"]}
                        for row in result]
            # Capture any errors along with the query and data for traceability
            except ServiceUnavailable as exception:
                logging.error("{query} raised an error: \n {exception}".format(
                    query=query, exception=exception))
                raise

    #Vishnu: 1 Aug 2022: created this f() to add use case labels to existing nodes
    #                    Note that the existing labels are left there.
    @staticmethod
    def _add_usecase_and_return_existing_node_label(tx, actor_name, usecase_id):
        query = (
            "MATCH "
            "(n1) "
            "WHERE n1.name = '" + actor_name +"' "
            "set n1 :" + usecase_id +" "
            "return n1"
        )
        result = tx.run(query, actor_name=actor_name)
        try:
            return [{"n1": row["n1"]["name"]}
                    for row in result]
        # Capture any errors along with the query and data for traceability
        except ServiceUnavailable as exception:
            logging.error("{query} raised an error: \n {exception}".format(
                query=query, exception=exception))
            raise

    #Vishnu: 1 Aug 2022: created this f() to return all use case actors based on labels
    #                    as against properties
    @staticmethod
    def _find_and_return_all_usecase_actors_label(tx, usecase_id):
        query = (
            "MATCH (n) "
            "WHERE '" +usecase_id+ "' in labels(n) "
            "RETURN n.name AS name"
        )
        result = tx.run(query)
        return [row["name"] for row in result]

    #Vishnu: 1 Aug 2022: created this f() to list all nodes with usecase label
    #                    instead of properties
    def find_all_usecase_actors_label(self, usecase_id):
        with self.driver.session() as session:
            result = session.read_transaction(self._find_and_return_all_usecase_actors_label, usecase_id)
            i=0
            for row in result:
                i=i+1
                print("Found actor-"+ str(i) +": {row}".format(row=row))

    #Vishnu: 1 Aug 2022: Created to use label for use case.
    #CAUTION: Assumes that actors already exists with label as use case.
    #So, this has to be called only in conjunction with create_node_with_usecase
    #otherwise you may end up with actors not having use case prop
    #but rels will have use case prop. not a good idea.
    def create_rel_with_usecase_label(self, actor1_name, rel_name, actor2_name, usecase_id):
        with self.driver.session() as session:
            this_rel_already_exists = session.read_transaction(
                self._find_and_return_this_existing_rels, actor1_name, rel_name, actor2_name, usecase_id)
            if (this_rel_already_exists):
                print(rel_name + " already exists with "+ usecase_id + " between "+ actor1_name + " and "+ actor2_name)
                return
            else:
                #we are pretty sure something didnt match
                #this relation does not exist currently between the nodes under the use case, 
                #add it.
                result3 = session.write_transaction(
                    self._create_and_return_rel_label, actor1_name, rel_name, actor2_name, usecase_id)
                #if (not retult3)
                for row in result3:
                    print("Created relation: {actor1} - {rel} - {actor2} in {usecase}".
                            format(actor1=row['n1name'], 
                            rel=row['r1name'],
                            actor2=row['n2name'],
                            usecase=row['r1usecase']))

    #Vishnu: 1 Aug 2022 
    #assumption: nodes already exist
    #modified from _create_and_return_rel to use labels for n.usecase
    @staticmethod
    def _create_and_return_rel_label(tx, actor1_name, rel_name, actor2_name, usecaseid):
            query = (
            "MATCH "
            "(n1), "
            "(n2) "
            "WHERE n1.name = '" + actor1_name + "' AND n2.name = '"+actor2_name+"' "
            " AND '"+ usecaseid+"' in labels(n1) "
            " AND '"+ usecaseid+"' in labels(n2) "
            "CREATE (n1)-[r1:Relation " + "{name: '"+rel_name+"', usecase: '"+usecaseid+"'} ]->(n2) "
            "RETURN n1, r1, n2"
            )
            result = tx.run(query)
            try:
                return [{"n1name": row["n1"]["name"], 
                        "r1name": row["r1"]["name"],
                        "n2name": row["n2"]["name"],
                        "r1usecase": row["r1"]["usecase"]}
                        for row in result]
            # Capture any errors along with the query and data for traceability
            except ServiceUnavailable as exception:
                logging.error("{query} raised an error: \n {exception}".format(
                    query=query, exception=exception))
                raise

    #Vishnu: 1 Aug 2022: This approach uses properties instead of labels for n.usecases.
    #Vishnu: 1 Aug 2022: Invariant: only 1 actor with 1 name.
    #Vishnu: 22 Jun 2022: refactored, uses properties instead of labels
    #Vishnu: 1 Jun 2022: created this f() to create nodes with usecase label
    #Vishnu: 2 Jun 2022: added- check if it already exists.
    #Vishnu: 2 Jun 2022: added- check the usecase duplication, adds a usecase if it doesnt exist.
    #otherwise ignores the command.
    def create_node_with_usecase(self, actor_name, usecase_id):
        with self.driver.session() as session:
            node_already_exists = session.read_transaction(
                self._find_and_return_existing_node, actor_name)
            if not node_already_exists:  
                result = session.write_transaction(
                    self._create_and_return_node, actor_name, usecase_id)
                for row in result:
                    print("Created node: {n1}".format(n1=row['n1name']))
            else:
                for row in node_already_exists:
                    print("Node already exists: {name} in {usecase}".format(name=row['n1name'], 
                                                                 usecase=row['n1usecase']))
                    s=row['n1usecase'].split(',')
                    if not (usecase_id in s):
                      print("adding prop "+usecase_id+ " in addition to "+row['n1usecase'])
                      result = session.write_transaction(
                                self._add_usecase_and_return_existing_node, actor_name, usecase_id)
                    else:
                      print("ignoring ...")
                    #row has only 1 entry due to invariant. So break here. and exit f()
                    break

    #Vishnu: 1 Aug 2022: This approach uses properties instead of labels for usecases.
    #Vishnu: 2 Jun 2022: created this f() to query nodes and return with usecase prop
    @staticmethod
    def _find_and_return_existing_node(tx, actor_name):
        query = (
            "MATCH (n1) "
            "WHERE (n1.name ='" + actor_name + "') "
            "RETURN n1.name as n1name, n1.usecase as n1usecase"
        )
        result = tx.run(query)
        try:
            return [{"n1name": row["n1name"],"n1usecase": row["n1usecase"]} 
                    for row in result]
        # Capture any errors along with the query and data for traceability
        except ServiceUnavailable as exception:
            logging.error("{query} raised an error: \n {exception}".format(
                query=query, exception=exception))
            raise

    #Vishnu: 1 Aug 2022: This approach uses properties instead of labels for usecases.
    @staticmethod
    def _create_and_return_node(tx, actor_name, usecase_id):
            query = (
            "CREATE (n1 "+"{ name: '"+actor_name+"', usecase: '"+usecase_id+"'}) "
            "RETURN n1"
            )
            result = tx.run(query, actor_name=actor_name)
            try:
                return [{"n1name": row["n1"]["name"]}
                        for row in result]
            # Capture any errors along with the query and data for traceability
            except ServiceUnavailable as exception:
                logging.error("{query} raised an error: \n {exception}".format(
                    query=query, exception=exception))
                raise

    #Vishnu: 1 Aug 2022: This approach uses properties instead of labels for usecases.
    #Vishnu: 1 Jun 2022: created this f() to add use case prop to existing nodes 
    #CAUTION: does not check for duplication, use create_node_with_prop instead.
    def add_usecase_existing_node(self, actor_name, usecase_id):
        with self.driver.session() as session:
            # Write transactions allow the driver to handle retries and transient errors
            result = session.write_transaction(
                self._add_usecase_and_return_existing_node, actor_name, usecase_id)
            
            # we expect only 1 node. Do we need the for loop below?
            for row in result:
                print("added usecase to node: {n1}".format(n1=row['n1']))

    #Vishnu: 1 Aug 2022: This approach uses properties instead of labels for usecases.
    @staticmethod
    def _add_usecase_and_return_existing_node(tx, actor_name, usecase_id):
        # To learn more about the Cypher syntax, see https://neo4j.com/docs/cypher-manual/current/
        # The Reference Card is also a good resource for keywords https://neo4j.com/docs/cypher-refcard/current/
        query = (
            "MATCH "
            "(n1) "
            "WHERE n1.name = '" + actor_name +"' "
            "set n1.usecase = n1.usecase + '" + "," + "'"+ "+ '"+usecase_id +"' "
            "return n1"
        )
        result = tx.run(query, actor_name=actor_name)
        try:
            return [{"n1": row["n1"]["name"]}
                    for row in result]
        # Capture any errors along with the query and data for traceability
        except ServiceUnavailable as exception:
            logging.error("{query} raised an error: \n {exception}".format(
                query=query, exception=exception))
            raise

    #Vishnu: 1 Aug 2022: This approach uses properties instead of labels for usecases.
    #CAUTION: Assumes that actors already exists with property as n.use case.
    #So, this has to be called only in conjunction with create_node_with_usecase
    #otherwise you may end up with actors not having use case prop
    #but rels will have use case prop. not a good idea.
    def create_rel_with_usecase(self, actor1_name, rel_name, actor2_name, usecase_id):
        with self.driver.session() as session:
            this_rel_already_exists = session.read_transaction(
                self._find_and_return_this_existing_rels, actor1_name, rel_name, actor2_name, usecase_id)
            if (this_rel_already_exists):
                print(rel_name + " already exists with "+ usecase_id + " between "+ actor1_name + " and "+ actor2_name)
                return
            else:
                #we are pretty sure something didnt match
                #this relation does not exist currently between the nodes under the use case, 
                #add it.
                result3 = session.write_transaction(
                    self._create_and_return_rel, actor1_name, rel_name, actor2_name, usecase_id)
                #if (not retult3)
                for row in result3:
                    print("Created relation: {actor1} - {rel} - {actor2} in {usecase}".
                            format(actor1=row['n1name'], 
                            rel=row['r1name'],
                            actor2=row['n2name'],
                            usecase=row['r1usecase']))
    
    #Vishnu: 1 Aug 2022: at this moment this f() uses labels instead of properties for n.usecases.
    #                    Note that rels still uses properties.
    #        1 Aug 2022: TBD: create a property-version (as against label) of the same thing.
    #Vishnu: 21 Jun 2022: created this f() to create relationships with usecase label
    #if this works, this is the only function that needed to be called.
    def create_actors_relationship_with_usecase(self, actor1_name, rel_name, actor2_name, usecase_id):
            #process actor1
            self.create_node_with_usecase_label(actor1_name, usecase_id)
            #process actor2
            self.create_node_with_usecase_label(actor2_name, usecase_id)
            #process relation
            self.create_rel_with_usecase_label(actor1_name, rel_name, actor2_name, usecase_id)

    #Vishnu: 23 June 2022: created
    #looks for a rel under a use case between 2 given nodes.
    #normally, we expect only 0 or 1 unique entry.
    @staticmethod                
    def _find_and_return_this_existing_rels(tx, actor1_name, rel_name, actor2_name, usecase_id):
        query = (
            "MATCH (n1)-[r1]-(n2) "
            "WHERE (n1.name ='" + actor1_name + "') "
            "and   (n2.name ='" + actor2_name + "') "
            "and   (r1.usecase CONTAINS '"+ usecase_id+ "') "
            "and   (r1.name ='" + rel_name + "') "
            "RETURN n1.name as n1name, r1.name as r1name, n2.name as n2name, r1.usecase as r1usecase"
        )
        result = tx.run(query)
        return [row["r1name"] for row in result]
        
    
    #Vishnu: 21 June 2022: created this supporting f()
    #return for all relationship between given two nodes.
    @staticmethod                
    def _find_and_return_all_existing_rels_for_uc(tx, actor1_name, actor2_name, usecase_id):
        query = (
            "MATCH (n1)-[r1]-(n2) "
            "WHERE (n1.name ='" + actor1_name + "') "
            "and   (n2.name ='" + actor2_name + "') "
            "and   (r1.usecase CONTAINS '"+ usecase_id+ "') "
            "RETURN n1.name as n1name, r1.name as r1name, n2.name as n2name, r1.usecase as r1usecase"
        )
        result = tx.run(query)
        try:
            return [{
                        "n1name": row["n1name"],
                        "n2name": row["n2name"],
                        "r1name": row["r1name"],
                        "r1usecase": row["r1usecase"],
                    } 
                    for row in result]
        # Capture any errors along with the query and data for traceability
        except ServiceUnavailable as exception:
            logging.error("{query} raised an error: \n {exception}".format(
                query=query, exception=exception))
            raise

    #Vishnu: 21 June 2022: created this supporting f()
    #adds a name for an existing relationship between given two nodes.
    @staticmethod                
    def _add_usecase_and_return_existing_rel(tx, actor1_name, rel_name, actor2_name, usecase_id):
        query = (
            "MATCH (n1)-[r1]-(n2) "
            "WHERE (n1.name ='" + actor1_name + "') "
            "and   (n2.name ='" + actor2_name + "')"
            "set    r1.usecase = r1.usecase + '" + "," + "'"+ "+ '"+usecase_id +"' "
        )
        result = tx.run(query)

    
    #Vishnu: 30 june 2022: created
    #CAUTION: if the property already exists, 
    #this overwrites the current value.
    @staticmethod
    def _write_property_and_return_existing_node(tx, actor_name, propertyName, propertyVal):
        query = (
            "MATCH "
            "(n1) "
            "WHERE n1.name = '" + actor_name +"' "
            "set n1."+propertyName+" = '"+propertyVal +"' "
            "return n1"
        )
        result = tx.run(query, actor_name=actor_name)
        try:
            return [{"n1": row["n1"]["name"]}
                    for row in result]
        # Capture any errors along with the query and data for traceability
        except ServiceUnavailable as exception:
            logging.error("{query} raised an error: \n {exception}".format(
                query=query, exception=exception))
            raise

    #Vishnu: 30 june 2022: created
    #CAUTION: if the property already exists, 
    #this overwrites the current value.
    @staticmethod
    def _write_property_and_return_existing_rel(tx, actor1_name, rel_name, actor2_name, 
                                                propertyName, propertyVal):
        query = (
            "MATCH (n1)-[r1]-(n2) "
            "WHERE (n1.name ='" + actor1_name + "') "
            "and   (n2.name ='" + actor2_name + "') "
            "and   (r1.name ='" + rel_name + "') "
            "set   r1."+propertyName+" = '"+propertyVal +"' "
            "return r1"
        )
        result = tx.run(query)
        try:
            return [{"r1": row["r1"]["name"]}
                    for row in result]
        # Capture any errors along with the query and data for traceability
        except ServiceUnavailable as exception:
            logging.error("{query} raised an error: \n {exception}".format(
                query=query, exception=exception))
            raise

    #Vishnu: 22 June 2022: created
    #assumption: nodes already exist
    @staticmethod
    def _create_and_return_rel(tx, actor1_name, rel_name, actor2_name, usecaseid):
            query = (
            "MATCH "
            "(n1), "
            "(n2) "
            "WHERE n1.name = '" + actor1_name + "' AND n2.name = '"+actor2_name+"' "
            " AND n1.usecase CONTAINS '"+ usecaseid+"' "
            " AND n2.usecase CONTAINS '"+ usecaseid+"' "
            "CREATE (n1)-[r1:Relation " + "{name: '"+rel_name+"', usecase: '"+usecaseid+"'} ]->(n2) "
            "RETURN n1, r1, n2"
            )
            result = tx.run(query)
            try:
                return [{"n1name": row["n1"]["name"], 
                        "r1name": row["r1"]["name"],
                        "n2name": row["n2"]["name"],
                        "r1usecase": row["r1"]["usecase"]}
                        for row in result]
            # Capture any errors along with the query and data for traceability
            except ServiceUnavailable as exception:
                logging.error("{query} raised an error: \n {exception}".format(
                    query=query, exception=exception))
                raise

    #Vishnu: 1 Jun 2022: created this f() to list all nodes with usecase label
    def find_all_usecase_actors(self, usecase_id):
        with self.driver.session() as session:
            result = session.read_transaction(self._find_and_return_all_usecase_actors, usecase_id)
            i=0
            for row in result:
                i=i+1
                print("Found actor-"+ str(i) +": {row}".format(row=row))

    @staticmethod
    def _find_and_return_all_usecase_actors(tx, usecase_id):
        query = (
            "MATCH (n) "
            "WHERE (n.usecase = '"+usecase_id+"') "
            "RETURN n.name AS name"
        )
        result = tx.run(query)
        return [row["name"] for row in result]

    #Vishnu: 21 Jun 2022: created this f() to cleanup the DB
    #CAUTION: will delete everything
    def cleanup_db(self):
        with self.driver.session() as session:
            result = session.read_transaction(
                self._count_nodes_in_db)
            for row in result:
                print("Found {row} actors, deleting! ".format(row=row['count']))
            result = session.write_transaction(
                self._cleanup_db)

    #Vishnu: 22 June 2022: created this f() 
    #(over)writes a prop to a node
    def writePropToNode(self, actor_name, propertyName, propertyVal):
        with self.driver.session() as session:
            result = session.write_transaction(
                self._write_property_and_return_existing_node, 
                    actor_name, 
                    propertyName, 
                    propertyVal)
        
            # we expect only 1 node. Do we need the for loop below?
            for row in result:
                print("added prop "+
                        propertyName+
                        " = "+
                        propertyVal+
                        " to node: {n1}".format(n1=row['n1']))

    #Vishnu: 22 June 2022: created this f() 
    #(over)writes a prop to a rel
    def writePropToRel(self, a1, rel_name, a2, propertyName, propertyVal):
        with self.driver.session() as session:
            result = session.write_transaction(
                self._write_property_and_return_existing_rel, 
                    a1, rel_name, a2,
                    propertyName, 
                    propertyVal)
        
            # we expect only 1 node. Do we need the for loop below?
            for row in result:
                print("added prop "+
                        propertyName+
                        " = "+
                        propertyVal+
                        " to rel: {r1}".format(r1=row['r1']))

    @staticmethod
    def _cleanup_db(tx):
        query = (
            "MATCH (n) "
            "detach delete n"
        )
        result = tx.run(query)
        try:
            return 
        # Capture any errors along with the query and data for traceability
        except ServiceUnavailable as exception:
            logging.error("{query} raised an error: \n {exception}".format(
                query=query, exception=exception))
            raise

    @staticmethod
    def _count_nodes_in_db(tx):
        query = (
            "MATCH (n) "
            "return count(n) as count"
        )
        result = tx.run(query)
        try:
            return [{"count": row["count"]} 
                    for row in result]
        # Capture any errors along with the query and data for traceability
        except ServiceUnavailable as exception:
            logging.error("{query} raised an error: \n {exception}".format(
                query=query, exception=exception))
            raise

## step-4: instantiate the helper class to DB. 
CAUTION: this uses the username/passwd assigned already in step-2 above.

In [8]:
app = App(uri, user, password)

## step-5 clean up DB
#Caution: lose data


In [ ]:
app.cleanup_db()

## Examples

In [ ]:
app.create_actors_relationship_with_usecase("Experimentation Controller", 
                                            "myrel", 
                                            "Sandbox",
                                            "usecase_001")

### Example-1: Create individual nodes with usecase
(and add a node property)
(and add a rel property)

In [ ]:
app.create_node_with_usecase("Experimentation Controller", "usecase_001")
app.create_node_with_usecase("Sandbox", "usecase_001")


In [ ]:
app.writePropToNode("Sandbox", "capability", "import knowledge")


In [ ]:
app.writePropToNode("Sandbox", "requirement", "integrate controllers")


### Example-1a: try to create duplicate node under different use case

In [ ]:
app.create_node_with_usecase("Sandbox", "usecase_003")
app.create_node_with_usecase("Sandbox", "usecase_004")
app.create_node_with_usecase("Sandbox", "usecase_005")
app.create_node_with_usecase("Sandbox", "usecase_006")


### Example-1b: use create_node_with_usecase_label (instead of properties)

In [ ]:
app.create_node_with_usecase_label("Sandbox", "usecase_003")
app.create_node_with_usecase_label("Sandbox", "usecase_003")
app.create_node_with_usecase_label("Sandbox", "usecase_004")
app.create_node_with_usecase_label("Sandbox", "usecase_004")
app.create_node_with_usecase_label("Sandbox", "usecase_005")



### Example-2: Create relation between 2 existing nodes

In [ ]:
app.create_rel_with_usecase("Experimentation Controller", "my_best_buddy", "Sandbox", "usecase_001")
app.create_rel_with_usecase("Experimentation Controller", "my_best_buddy", "Sandbox", "usecase_002")
app.create_rel_with_usecase("Experimentation Controller", "my_best_buddy", "Sandbox", "usecase_003")


In [ ]:
app.writePropToRel("Experimentation Controller", "my_best_buddy", "Sandbox", 
                    "my_rel_prop_name", 
                    "my-rel-prop-val")

## Cleanup DB 
CAUTION: Remember to clean up the DB before executing the next steps. (so that all examples above are not leftover in the use case data below).

In [ ]:
app.cleanup_db()

## USECASES

### FG-AN-USECASE-001

In [ ]:

app.create_actors_relationship_with_usecase("Knowledge Base", "import", "Knowledge Base", "usecase_001")
app.create_actors_relationship_with_usecase("Knowledge Base", "export", "Knowledge Base", "usecase_001")

app.create_actors_relationship_with_usecase("Knowledge Base Manager", "optimizes", "Knowledge Base", "usecase_001")
app.create_actors_relationship_with_usecase("Knowledge Base Manager", "export", "Knowledge Base", "usecase_001")

app.create_actors_relationship_with_usecase("AN Orchestrator", "refer", "Knowledge Base", "usecase_001")
app.create_actors_relationship_with_usecase("AN Orchestrator", "input", "Use case description", "usecase_001")
app.create_actors_relationship_with_usecase("AN Orchestrator", "output", "TOSCA format", "usecase_001")

app.create_actors_relationship_with_usecase("Auto controller generator", "refer", "Knowledge Base", "usecase_001")
app.create_actors_relationship_with_usecase("Auto controller generator", "output", "TOSCA format", "usecase_001")

app.create_actors_relationship_with_usecase("controller repository", "stores", "controllers", "usecase_001")
app.create_actors_relationship_with_usecase("ML model", "recommends", "controllers", "usecase_001")

app.create_actors_relationship_with_usecase("Human operator", "input", "reports", "usecase_001")
app.create_actors_relationship_with_usecase("Human operator", "monitors", "reports", "usecase_001")

In [ ]:
app.find_all_usecase_actors_label("usecase_001")

### FG-AN-USECASE-002


In [ ]:
app.create_actors_relationship_with_usecase("Experimentation controller", "decides", 
                                        "scenarios for experimentation", "usecase_002")
app.create_actors_relationship_with_usecase("AN sandbox", "hosts", 
                                        "simulators", "usecase_002")
app.create_actors_relationship_with_usecase("AN sandbox", "manages", 
                                        "simulators", "usecase_002")
app.create_actors_relationship_with_usecase("AN sandbox", "output", 
                                        "reports", "usecase_002")
app.create_actors_relationship_with_usecase("Experimentation controller", "triggers", 
                                        "experiments", "usecase_002")
app.create_actors_relationship_with_usecase("Experimentation controller", "output", 
                                        "reports", "usecase_002")
app.create_actors_relationship_with_usecase("Experimentation controller", "executes", 
                                        "experiments", "usecase_002")
app.create_actors_relationship_with_usecase("Experimentation controller", "monitors", 
                                        "experiments", "usecase_002")
app.create_actors_relationship_with_usecase("Experimentation controller", "uses", 
                                        "AN sandbox", "usecase_002")
app.create_actors_relationship_with_usecase("Experimentation controller", "updates", 
                                        "Knowledge Base", "usecase_002")
app.create_actors_relationship_with_usecase("experiments", "has", 
                                        "scenarios for experimentation", "usecase_002")

In [ ]:
app.find_all_usecase_actors_label("usecase_002")

### FG-AN-USECASE-003


In [ ]:
app.create_actors_relationship_with_usecase("AN Orchestrator", "queries", 
                                        "AN Orchestrator", "usecase_003")
app.create_actors_relationship_with_usecase("AN Orchestrator", "queries", 
                                        "Human operator", "usecase_003")
app.create_actors_relationship_with_usecase("AN Orchestrator", "accept", 
                                        "feedback", "usecase_003")
app.create_actors_relationship_with_usecase("AN Orchestrator", "configures", 
                                        "AN components", "usecase_003")
app.create_actors_relationship_with_usecase("AN Orchestrator", "updates", 
                                        "Knowledge Base", "usecase_003")


In [ ]:
app.find_all_usecase_actors_label("usecase_003")

### FG-AN-USECASE-004

In [ ]:
app.create_actors_relationship_with_usecase("Adaptation controller", "discovers", 
                                        "controllers", "usecase_004")
app.create_actors_relationship_with_usecase("Adaptation controller", "deploy", 
                                        "controllers", "usecase_004")
app.create_actors_relationship_with_usecase("Adaptation controller", "configures", 
                                        "controllers", "usecase_004")
app.create_actors_relationship_with_usecase("Adaptation controller", "queries", 
                                        "controller repository", "usecase_004")          
app.create_actors_relationship_with_usecase("Adaptation controller", "updates", 
                                        "Knowledge Base", "usecase_004")  
app.create_actors_relationship_with_usecase("controllers", "is", 
                                        "undeployed controllers", "usecase_004")  
app.create_actors_relationship_with_usecase("controllers", "is", 
                                        "deployed controllers", "usecase_004")                                         

In [ ]:
app.find_all_usecase_actors_label("usecase_004")

### FG-AN-USECASE-005

In [ ]:
app.create_actors_relationship_with_usecase("Service management framework", "is", 
                                        "underlay", "usecase_005")
app.create_actors_relationship_with_usecase("Adaptation controller", "discover", 
                                        "Service management framework", "usecase_005")     
app.create_actors_relationship_with_usecase("Adaptation controller", "influence", 
                                        "Service management framework", "usecase_005")  
app.create_actors_relationship_with_usecase("intent", "is", 
                                        "influence", "usecase_005")  
app.create_actors_relationship_with_usecase("policy", "is", 
                                        "influence", "usecase_005")                                         

In [ ]:
app.find_all_usecase_actors_label("usecase_005")

### FG-AN-USECASE-006 cat1

In [ ]:
app.create_actors_relationship_with_usecase("Evolution controller", "monitors", 
                                        "controllers", "usecase_006_cat1")
app.create_actors_relationship_with_usecase("Evolution controller", "discovers", 
                                        "use cases", "usecase_006_cat1")     
app.create_actors_relationship_with_usecase("Evolution controller", "recommends", 
                                        "controllers", "usecase_006_cat1")  
app.create_actors_relationship_with_usecase("Evolution controller", "refers", 
                                        "Knowledge Base", "usecase_006_cat1")  

In [ ]:
app.find_all_usecase_actors_label("usecase_006_cat1")

### FG-AN-USECASE-006 cat2

In [ ]:
app.create_actors_relationship_with_usecase("Data collector", "consumes", 
                                        "data", "usecase_006_cat2")
app.create_actors_relationship_with_usecase("Management domain", "produces", 
                                        "data", "usecase_006_cat2")     
app.create_actors_relationship_with_usecase("Management domain", "has", 
                                        "RAN cell", "usecase_006_cat2")  
app.create_actors_relationship_with_usecase("Data collector", "produces", 
                                        "data", "usecase_006_cat2") 

app.create_actors_relationship_with_usecase("data", "has", 
                                        "real time", "usecase_006_cat2")     
app.create_actors_relationship_with_usecase("data", "has", 
                                        "data quality", "usecase_006_cat2")  
app.create_actors_relationship_with_usecase("data", "has", 
                                        "KPI", "usecase_006_cat2") 

app.create_actors_relationship_with_usecase("Data monitor", "consumes", 
                                        "data", "usecase_006_cat2")     
app.create_actors_relationship_with_usecase("Data monitor", "has", 
                                        "data monitoring", "usecase_006_cat2")  
app.create_actors_relationship_with_usecase("Data monitor", "produces", 
                                        "reports", "usecase_006_cat2") 
app.create_actors_relationship_with_usecase("reports", "has", 
                                        "visualizations", "usecase_006_cat2")                                         
app.create_actors_relationship_with_usecase("Data analytics", "consumes", 
                                        "data", "usecase_006_cat2")    
app.create_actors_relationship_with_usecase("Data analytics", "produces", 
                                        "inference", "usecase_006_cat2")                                           
app.create_actors_relationship_with_usecase("Data analytics", "has", 
                                        "ML model", "usecase_006_cat2") 
app.create_actors_relationship_with_usecase("inference", "has", 
                                        "periodicity", "usecase_006_cat2")      
app.create_actors_relationship_with_usecase("ML model", "has", 
                                        "model training", "usecase_006_cat2")

In [ ]:
app.find_all_usecase_actors_label("usecase_006_cat2")

### FG-AN-USECASE-007 cat1

In [ ]:
app.create_actors_relationship_with_usecase("Adaptation controller", "discovers deployed", 
                                        "CL automation framework", "usecase_007_cat1")
app.create_actors_relationship_with_usecase("CL automation framework", "deploys", 
                                        "controllers", "usecase_007_cat1")
app.create_actors_relationship_with_usecase("controllers", "has", 
                                        "automatic data acquisition", "usecase_007_cat1")
app.create_actors_relationship_with_usecase("controllers", "has", 
                                        "ML model", "usecase_007_cat1")
app.create_actors_relationship_with_usecase("controllers", "has", 
                                        "policy", "usecase_007_cat1")
app.create_actors_relationship_with_usecase("controllers", "has", 
                                        "action implementation", "usecase_007_cat1")
app.create_actors_relationship_with_usecase("policy", "need", 
                                        "optimization", "usecase_007_cat1")
app.create_actors_relationship_with_usecase("Adaptation controller", "configures", 
                                        "controllers", "usecase_007_cat1")
app.create_actors_relationship_with_usecase("Adaptation controller", "monitors", 
                                        "controllers", "usecase_007_cat1")                                       
app.create_actors_relationship_with_usecase("Adaptation controller", "adapts", 
                                        "controllers", "usecase_007_cat1") 
app.create_actors_relationship_with_usecase("Adaptation controller", "interfaces with", 
                                        "underlay orchestrator", "usecase_007_cat1")   

In [ ]:
app.find_all_usecase_actors_label("usecase_007_cat1")

### FG-AN-USECASE-007 cat2

In [ ]:
app.create_actors_relationship_with_usecase("Energy saving controller", "uses", 
                                        "modules", "usecase_007_cat2")
app.create_actors_relationship_with_usecase("controllers", "has", 
                                        "modules", "usecase_007_cat2")    
app.create_actors_relationship_with_usecase("data acquisition", "is", 
                                        "modules", "usecase_007_cat2")
app.create_actors_relationship_with_usecase("policy", "is", 
                                        "modules", "usecase_007_cat2")  
app.create_actors_relationship_with_usecase("Action implementation", "is", 
                                        "modules", "usecase_007_cat2")        
app.create_actors_relationship_with_usecase("Data acquisition", "uses", 
                                        "data center APIs", "usecase_007_cat2")   
app.create_actors_relationship_with_usecase("policy", "has", 
                                        "data center policy", "usecase_007_cat2")   
app.create_actors_relationship_with_usecase("Action implementation", "uses", 
                                        "data center APIs", "usecase_007_cat2")        
app.create_actors_relationship_with_usecase("controllers", "need", 
                                        "evaluation", "usecase_007_cat2")                
app.create_actors_relationship_with_usecase("evaluation", "has", 
                                        "metrics", "usecase_007_cat2")        
app.create_actors_relationship_with_usecase("metrics", "are", 
                                        "data center metrics", "usecase_007_cat2") 
app.create_actors_relationship_with_usecase("controllers", "need", 
                                        "optimization", "usecase_007_cat2")     
app.create_actors_relationship_with_usecase("optimization", "uses", 
                                        "evaluation", "usecase_007_cat2")                                                

In [ ]:
app.find_all_usecase_actors_label("usecase_007_cat2")

### FG-AN-USECASE-008 cat1

In [ ]:
app.create_actors_relationship_with_usecase("CL automation framework", "discovers", "network parameters", "usecase_008_cat1")
app.create_actors_relationship_with_usecase("CL automation framework", "discovers", "data", "usecase_008_cat1")
app.create_actors_relationship_with_usecase("CL automation framework", "discovers", "KPI", "usecase_008_cat1")
app.create_actors_relationship_with_usecase("CL automation framework", "deploys", "controllers", "usecase_008_cat1")
app.create_actors_relationship_with_usecase("controllers", "provide", "optimization", "usecase_008_cat1")
app.create_actors_relationship_with_usecase("KPI", "need", "optimization", "usecase_008_cat1")
app.create_actors_relationship_with_usecase("use cases", "has", "KPI", "usecase_008_cat1")
          

In [ ]:
app.create_actors_relationship_with_usecase("Evolution controller", "discovers", "use cases", "usecase_008_cat1")
app.create_actors_relationship_with_usecase("Evolution controller", "composes", "controllers", "usecase_008_cat1")
app.create_actors_relationship_with_usecase("use cases", "has", "data", "usecase_008_cat1")
app.create_actors_relationship_with_usecase("data", "has", "parameters", "usecase_008_cat1")


In [ ]:
app.find_all_usecase_actors_label("usecase_008_cat1")

### FG-AN-USECASE-008 cat2

In [ ]:
app.create_actors_relationship_with_usecase("base station", "has", "UE location", "usecase_008_cat2")
app.create_actors_relationship_with_usecase("mMiMO controller", "needs", "UE location", "usecase_008_cat2")
app.create_actors_relationship_with_usecase("mMiMO controller", "has", "optimization", "usecase_008_cat2") 
app.create_actors_relationship_with_usecase("optimization", "has", "goals", "usecase_008_cat2")
app.create_actors_relationship_with_usecase("goals", "are", "system utilization", "usecase_008_cat2")
app.create_actors_relationship_with_usecase("goals", "are", "user experience", "usecase_008_cat2")
app.create_actors_relationship_with_usecase("goals", "are", "target RSRP/SINR distribution", "usecase_008_cat2")
app.create_actors_relationship_with_usecase("goals", "are", "optimal antenna parameters", "usecase_008_cat2")

In [ ]:
app.find_all_usecase_actors_label("usecase_008_cat2")

### FG-AN-USECASE-009 cat1

In [ ]:
app.create_actors_relationship_with_usecase("Adaptation controller", "discovers", "controller characteristics", "usecase_009_cat1")
app.create_actors_relationship_with_usecase("Evolution controller", "accepts", "AN intent", "usecase_009_cat1")
app.create_actors_relationship_with_usecase("Evolution controller", "composes", "controllers", "usecase_009_cat1")
app.create_actors_relationship_with_usecase("Evolution controller", "recommends", "controllers", "usecase_009_cat1")
app.create_actors_relationship_with_usecase("Evolution controller", "evolves", "controllers", "usecase_009_cat1")
app.create_actors_relationship_with_usecase("Evolution controller", "evolves", "modules", "usecase_009_cat1")
app.create_actors_relationship_with_usecase("Evolution controller", "implements", "evolution algorithms", "usecase_009_cat1")
app.create_actors_relationship_with_usecase("Evolution controller", "recommends", "modules", "usecase_009_cat1")
app.create_actors_relationship_with_usecase("Evolution controller", "composes", "controllers", "usecase_009_cat1")
app.create_actors_relationship_with_usecase("Adaptation controller", "deploys", "controllers", "usecase_009_cat1")
app.create_actors_relationship_with_usecase("controllers", "utilizes", "underlay specific APIs", "usecase_009_cat1")
app.create_actors_relationship_with_usecase("edge controllers", "are", "controllers", "usecase_009_cat1")


In [ ]:
app.find_all_usecase_actors_label("usecase_009_cat1")

### FG-AN-USECASE-009 cat2


In [ ]:
app.create_actors_relationship_with_usecase("Emergency management controller", "uses", "controllers", "usecase_009_cat2")
app.create_actors_relationship_with_usecase("controllers", "detect", "emergencies", "usecase_009_cat2")
app.create_actors_relationship_with_usecase("controllers", "optimize", "network resource allocation", "usecase_009_cat2")
app.create_actors_relationship_with_usecase("Emergency management controller", "instantiates", "controllers", "usecase_009_cat2")
app.create_actors_relationship_with_usecase("Emergency management controller", "input", "intent", "usecase_009_cat2")
app.create_actors_relationship_with_usecase("Emergency management controller", "output", "intent", "usecase_009_cat2")
app.create_actors_relationship_with_usecase("controllers", "need", "network domain", "usecase_009_cat2")
app.create_actors_relationship_with_usecase("intent", "has", "ml pipeline", "usecase_009_cat2")


In [ ]:
app.find_all_usecase_actors_label("usecase_009_cat2")

### FG-AN-USECASE-010 cat1

In [ ]:
app.create_actors_relationship_with_usecase("AN orchestrator", "accepts", "AN intent", "usecase_010_cat1")
app.create_actors_relationship_with_usecase("AN orchestrator", "configures", "Evolution controller", "usecase_010_cat1")
app.create_actors_relationship_with_usecase("AN orchestrator", "configures", "Adaptation controller", "usecase_010_cat1")
app.create_actors_relationship_with_usecase("AN orchestrator", "configures", "Experimentation controller", "usecase_010_cat1")
app.create_actors_relationship_with_usecase("AN intent", "has", "vertical specific inputs", "usecase_010_cat1")
app.create_actors_relationship_with_usecase("Experimentation controller", "validates", "controllers", "usecase_010_cat1")
app.create_actors_relationship_with_usecase("Adaptation controller", "monitors", "controllers", "usecase_010_cat1")
app.create_actors_relationship_with_usecase("AN orchestrator", "produces", "reports", "usecase_010_cat1")
app.create_actors_relationship_with_usecase("underlay", "are", "CI/CD pipeline", "usecase_010_cat1")
app.create_actors_relationship_with_usecase("Adaptation controller", "discovers", "reference points", "usecase_010_cat1")
app.create_actors_relationship_with_usecase("underlay", "has", "reference points", "usecase_010_cat1")
app.create_actors_relationship_with_usecase("Evolution controller", "recommends", "controllers", "usecase_010_cat1")
app.create_actors_relationship_with_usecase("Adaptation controller", "discovers", "underlay capabilities", "usecase_010_cat1")
app.create_actors_relationship_with_usecase("underlay", "has", "underlay capabilities", "usecase_010_cat1")



In [ ]:
app.find_all_usecase_actors_label("usecase_010_cat1")

### FG-AN-USECASE-010 cat2

In [ ]:
app.create_actors_relationship_with_usecase("Cloud service controller", "uses", "controllers", "usecase_010_cat2")
app.create_actors_relationship_with_usecase("controllers", "needs", "intent", "usecase_010_cat2")
app.create_actors_relationship_with_usecase("controllers", "needs", "simulation", "usecase_010_cat2")
app.create_actors_relationship_with_usecase("controllers", "needs", "validation", "usecase_010_cat2")
app.create_actors_relationship_with_usecase("validation", "needs", "KPI", "usecase_010_cat2")
app.create_actors_relationship_with_usecase("validation", "needs", "AN sandbox", "usecase_010_cat2")
app.create_actors_relationship_with_usecase("controllers", "output", "cloud service deployment", "usecase_010_cat2")
app.create_actors_relationship_with_usecase("cloud service deployment", "needs", "optimization", "usecase_010_cat2")
app.create_actors_relationship_with_usecase("controllers", "output", "network service deployment", "usecase_010_cat2")
app.create_actors_relationship_with_usecase("network service deployment", "needs", "optimization", "usecase_010_cat2")
app.create_actors_relationship_with_usecase("controllers", "needs", "monitors", "usecase_010_cat2")
app.create_actors_relationship_with_usecase("monitors", "output", "reports", "usecase_010_cat2")
app.create_actors_relationship_with_usecase("network service deployment", "needs", "technology migration", "usecase_010_cat2")
app.create_actors_relationship_with_usecase("cloud service deployment", "needs", "technology migration", "usecase_010_cat2")




In [ ]:
app.find_all_usecase_actors_label("usecase_010_cat2")

### FG-AN-USECASE-011 cat1

In [ ]:
app.create_actors_relationship_with_usecase("AN orchestrator", "input", "intent", "usecase_011_cat1")
app.create_actors_relationship_with_usecase("AN orchestrator", "configures", "domain orchestrator", "usecase_011_cat1")
app.create_actors_relationship_with_usecase("AN orchestrator", "configures", "Evolution controller", "usecase_011_cat1")
app.create_actors_relationship_with_usecase("AN orchestrator", "configures", "Adaptation controller", "usecase_011_cat1")

app.create_actors_relationship_with_usecase("intent", "has", "controller design", "usecase_011_cat1")
app.create_actors_relationship_with_usecase("intent", "has", "controller deployment", "usecase_011_cat1")
app.create_actors_relationship_with_usecase("intent", "has", "controller management", "usecase_011_cat1")

app.create_actors_relationship_with_usecase("controllers", "monitors", "inter-domain-connectivity", "usecase_011_cat1")
app.create_actors_relationship_with_usecase("controllers", "configures", "inter-domain-connectivity", "usecase_011_cat1")
app.create_actors_relationship_with_usecase("controllers", "manages", "inter-domain-connectivity", "usecase_011_cat1")

app.create_actors_relationship_with_usecase("Evolution controller", "implements", "evolution algorithms", "usecase_011_cat1")
app.create_actors_relationship_with_usecase("evolution algorithms", "implements", "evolution", "usecase_011_cat1")
app.create_actors_relationship_with_usecase("controllers", "need", "evolution", "usecase_011_cat1")

app.create_actors_relationship_with_usecase("underlay", "are", "inter-domain-connectivity", "usecase_011_cat1")
app.create_actors_relationship_with_usecase("Evolution controller", "recommends", "controllers", "usecase_011_cat1")

In [ ]:
app.find_all_usecase_actors_label("usecase_011_cat1")

### FG-AN-USECASE-011 cat2

In [ ]:
app.create_actors_relationship_with_usecase("inter-domain-connectivity", "has", "alternatives", "usecase_011_cat2")
app.create_actors_relationship_with_usecase("verticals", "provide", "service", "usecase_011_cat2")
app.create_actors_relationship_with_usecase("Rural sites", "requirement", "service", "usecase_011_cat2")
app.create_actors_relationship_with_usecase("verticals", "requirement", "inter-domain-connectivity", "usecase_011_cat2")
app.create_actors_relationship_with_usecase("Rural sites", "requirement", "inter-domain-connectivity", "usecase_011_cat2")
app.create_actors_relationship_with_usecase("inter-domain-connectivity", "is", "shared", "usecase_011_cat2")
app.create_actors_relationship_with_usecase("inter-domain-connectivity", "requirement", "edge compute", "usecase_011_cat2")
app.create_actors_relationship_with_usecase("inter-domain-connectivity", "requirement", "CL automation framework", "usecase_011_cat2")
app.create_actors_relationship_with_usecase("edge compute", "requirement", "CL automation framework", "usecase_011_cat2")
app.create_actors_relationship_with_usecase("edge compute", "has", "micro service", "usecase_011_cat2")
app.create_actors_relationship_with_usecase("alternatives", "has", "heterogeneous", "usecase_011_cat2")
app.create_actors_relationship_with_usecase("underlay orchestrator", "has", "controllers", "usecase_011_cat2")
app.create_actors_relationship_with_usecase("inter-domain-connectivity", "requirement", "management", "usecase_011_cat2")
app.create_actors_relationship_with_usecase("underlay orchestrator", "has", "onboarding", "usecase_011_cat2")
app.create_actors_relationship_with_usecase("applications", "requirement", "onboarding", "usecase_011_cat2")
app.create_actors_relationship_with_usecase("underlay orchestrator", "has", "monitoring", "usecase_011_cat2")
app.create_actors_relationship_with_usecase("applications", "requirement", "monitoring", "usecase_011_cat2")
app.create_actors_relationship_with_usecase("underlay orchestrator", "has", "report", "usecase_011_cat2")
app.create_actors_relationship_with_usecase("human operator", "requirement", "report", "usecase_011_cat2")

In [ ]:
app.find_all_usecase_actors_label("usecase_011_cat2")

### FG-AN-USECASE-012 cat1

In [ ]:
app.create_actors_relationship_with_usecase("Adaptation controller", "monitors", "underlay capabilities", "usecase_012_cat1")
app.create_actors_relationship_with_usecase("Adaptation controller", "discovers", "underlay capabilities", "usecase_012_cat1")
app.create_actors_relationship_with_usecase("underlay capabilities", "is", "underlay configuration", "usecase_012_cat1")
app.create_actors_relationship_with_usecase("underlay capabilities", "is", "monitoring", "usecase_012_cat1")
app.create_actors_relationship_with_usecase("Evolution controller", "capability", "recommendation", "usecase_012_cat1")
app.create_actors_relationship_with_usecase("recommendation", "is", "underlay capabilities", "usecase_012_cat1")

In [ ]:
app.find_all_usecase_actors_label("usecase_012_cat1")

### FG-AN-USECASE-012 cat2

In [ ]:
app.create_actors_relationship_with_usecase("Data analysis function", "capability", "data collector", "usecase_012_cat2")
app.create_actors_relationship_with_usecase("Network function", "capability", "event report", "usecase_012_cat2")
app.create_actors_relationship_with_usecase("event report", "is", "UE IP address change", "usecase_012_cat2")
app.create_actors_relationship_with_usecase("event report", "is", "UP path change", "usecase_012_cat2")
app.create_actors_relationship_with_usecase("event report", "is", "PDU Session Establishment", "usecase_012_cat2")
app.create_actors_relationship_with_usecase("event report", "is", "PDU Session Release", "usecase_012_cat2")
app.create_actors_relationship_with_usecase("Data analysis function", "capability", "Data analytics", "usecase_012_cat2")
app.create_actors_relationship_with_usecase("Data analytics", "is", "Load information of UPF", "usecase_012_cat2")
app.create_actors_relationship_with_usecase("event report", "is", "data", "usecase_012_cat2")

In [ ]:
app.find_all_usecase_actors_label("usecase_012_cat2")

### FG-AN-USECASE-013 cat1

In [ ]:

app.create_actors_relationship_with_usecase("Network function", "has", "underlay capabilities", "usecase_013_cat1")
app.create_actors_relationship_with_usecase("Adaptation controller", "monitors", "underlay capabilities", "usecase_013_cat1")
app.create_actors_relationship_with_usecase("Adaptation controller", "discovers", "underlay capabilities", "usecase_013_cat1")
app.create_actors_relationship_with_usecase("Adaptation controller", "discovers", "underlay requirements", "usecase_013_cat1")
app.create_actors_relationship_with_usecase("Evolution controller", "capability", "recommendation", "usecase_013_cat1")
app.create_actors_relationship_with_usecase("recommendation", "is", "underlay capabilities", "usecase_013_cat1")
app.create_actors_relationship_with_usecase("recommendation", "is", "underlay requirements", "usecase_013_cat1")

In [ ]:
app.find_all_usecase_actors_label("usecase_013_cat1")

### FG-AN-USECASE-013 cat2

In [ ]:
app.create_actors_relationship_with_usecase("controllers", "requirement", "provisioning", "usecase_013_cat2")
app.create_actors_relationship_with_usecase("provisioning", "is", "multi-domain", "usecase_013_cat2")
app.create_actors_relationship_with_usecase("provisioning", "requirement", "underlay capabilities", "usecase_013_cat2")
app.create_actors_relationship_with_usecase("network instance", "requirement", "plug and play", "usecase_013_cat2")
app.create_actors_relationship_with_usecase("network instance", "is", "Network function", "usecase_013_cat2")
app.create_actors_relationship_with_usecase("network instance", "is", "network slice", "usecase_013_cat2")
app.create_actors_relationship_with_usecase("network instance", "is", "network service", "usecase_013_cat2")
app.create_actors_relationship_with_usecase("network instance", "capability", "data collection", "usecase_013_cat2")
app.create_actors_relationship_with_usecase("controllers", "requirement", "bootstrap", "usecase_013_cat2")
app.create_actors_relationship_with_usecase("network infrastructure", "requirement", "bootstrap", "usecase_013_cat2")
app.create_actors_relationship_with_usecase("network service", "requirement", "bootstrap", "usecase_013_cat2")
app.create_actors_relationship_with_usecase("applications", "requirement", "bootstrap", "usecase_013_cat2")

In [ ]:
app.find_all_usecase_actors_label("usecase_013_cat2")

### FG-AN-USECASE-014

In [ ]:
app.create_actors_relationship_with_usecase("underlay", "are", "hybrid", "usecase_014")
app.create_actors_relationship_with_usecase("hybrid", "capability", "simulators", "usecase_014")
app.create_actors_relationship_with_usecase("hybrid", "capability", "test network", "usecase_014")
app.create_actors_relationship_with_usecase("hybrid", "capability", "real network", "usecase_014")
app.create_actors_relationship_with_usecase("multi-domain", "capability", "hybrid", "usecase_014")

In [ ]:
app.find_all_usecase_actors_label("usecase_014")

### FG-AN-USECASE-015

*detailed implementation of this UC may be provided by Team "Avengers"*

In [ ]:
app.create_actors_relationship_with_usecase("data", "is", "logs", "usecase_015")
app.create_actors_relationship_with_usecase("network instance", "capability", "logs", "usecase_015")
app.create_actors_relationship_with_usecase("model repository", "capability", "ML model", "usecase_015")
app.create_actors_relationship_with_usecase("model repository", "is", "3rd party", "usecase_015")


In [ ]:
app.find_all_usecase_actors_label("usecase_015")

### FG-AN-USECASE-016

In [ ]:
app.create_actors_relationship_with_usecase("underlay orchestrator", "process", "underlay intent", "usecase_016")
app.create_actors_relationship_with_usecase("underlay intent", "has", "network instance", "usecase_016")
app.create_actors_relationship_with_usecase("intent", "is", "underlay intent", "usecase_016")
app.create_actors_relationship_with_usecase("intent", "is", "AN intent", "usecase_016")
app.create_actors_relationship_with_usecase("underlay orchestrator", "interfaces", "AN orchestrator", "usecase_016")
app.create_actors_relationship_with_usecase("controllers", "is", "multi-domain", "usecase_016")
app.create_actors_relationship_with_usecase("underlay orchestrator", "manages", "network instance", "usecase_016")
app.create_actors_relationship_with_usecase("orchestrator", "export", "intent", "usecase_016")
app.create_actors_relationship_with_usecase("orchestrator", "import", "intent", "usecase_016")

app.create_actors_relationship_with_usecase("orchestrator", "is", "AN orchestrator", "usecase_016")
app.create_actors_relationship_with_usecase("orchestrator", "is", "underlay orchestrator", "usecase_016")
app.create_actors_relationship_with_usecase("AN intent", "has", "controller  definitions", "usecase_016")
app.create_actors_relationship_with_usecase("underlay intent", "has", "network instance definitions", "usecase_016")

In [ ]:
app.find_all_usecase_actors_label("usecase_016")

### FG-AN-USECASE-017

In [ ]:
app.create_actors_relationship_with_usecase("data", "is", "user experience", "usecase_017")
app.create_actors_relationship_with_usecase("data collection", "capability", "subjective tests", "usecase_017")
app.create_actors_relationship_with_usecase("user experience", "requirement", "subjective tests", "usecase_017")
app.create_actors_relationship_with_usecase("subjective tests", "capability", "user selection", "usecase_017")
app.create_actors_relationship_with_usecase("subjective tests", "capability", "location selection", "usecase_017")
app.create_actors_relationship_with_usecase("applications", "requirement", "subjective tests", "usecase_017")
app.create_actors_relationship_with_usecase("applications", "capability", "KPI", "usecase_017")
app.create_actors_relationship_with_usecase("underlay", "capability", "measurements", "usecase_017")
app.create_actors_relationship_with_usecase("subjective tests", "capability","user opinion", "usecase_017")


app.create_actors_relationship_with_usecase("QoE prediction", "is", "controllers", "usecase_017")
app.create_actors_relationship_with_usecase("QoE prediction", "requirement", "KPI", "usecase_017")
app.create_actors_relationship_with_usecase("QoE prediction", "capability", "optimization", "usecase_017")
app.create_actors_relationship_with_usecase("QoE prediction", "requirement", "user behavior", "usecase_017")
app.create_actors_relationship_with_usecase("QoE prediction", "requirement", "KPI", "usecase_017")
app.create_actors_relationship_with_usecase("QoE prediction", "requirement", "ML model", "usecase_017")
app.create_actors_relationship_with_usecase("QoE prediction", "requirement", "subjective tests", "usecase_017")

app.create_actors_relationship_with_usecase("ML model", "maps", "QoE metrics", "usecase_017")
app.create_actors_relationship_with_usecase("ML model", "maps", "measurements", "usecase_017")
app.create_actors_relationship_with_usecase("underlay orchestrator", "discover", "measurements", "usecase_017")


In [ ]:
app.find_all_usecase_actors_label("usecase_017")

### FG-AN-USECASE-018

In [ ]:
app.create_actors_relationship_with_usecase("AN intent", "has", "outer loop", "usecase_018")
app.create_actors_relationship_with_usecase("AN intent", "has", "deployment preferences", "usecase_018")
app.create_actors_relationship_with_usecase("AN orchestrator", "requires", "AN intent", "usecase_018")
app.create_actors_relationship_with_usecase("AN orchestrator", "derives", "sub-intent", "usecase_018")
app.create_actors_relationship_with_usecase("sub-intent", "is", "intent", "usecase_018")
app.create_actors_relationship_with_usecase("sub-intent", "has", "inner loop", "usecase_018")
app.create_actors_relationship_with_usecase("inner loop", "is", "controller", "usecase_018")
app.create_actors_relationship_with_usecase("outer loop", "is", "controller", "usecase_018")
app.create_actors_relationship_with_usecase("AN orchestrator", "optimizes", "intent", "usecase_018")
app.create_actors_relationship_with_usecase("AN orchestrator", "requires", "feedback", "usecase_018")
app.create_actors_relationship_with_usecase("Adaptation controller", "provides", "feedback", "usecase_018")

app.create_actors_relationship_with_usecase("outer loop", "has", "QoS QoE metrics", "usecase_018")
app.create_actors_relationship_with_usecase("outer loop", "has", "deployment preferences", "usecase_018")
app.create_actors_relationship_with_usecase("outer loop", "has", "ML model", "usecase_018")
app.create_actors_relationship_with_usecase("outer loop", "has", "cache policy", "usecase_018")
app.create_actors_relationship_with_usecase("outer loop", "has", "scaling policy", "usecase_018")
app.create_actors_relationship_with_usecase("outer loop", "has", "edge selection", "usecase_018")
app.create_actors_relationship_with_usecase("outer loop", "has", "storage configuration", "usecase_018")
app.create_actors_relationship_with_usecase("outer loop", "has", "API selection", "usecase_018")
app.create_actors_relationship_with_usecase("outer loop", "has", "concurrency selection", "usecase_018")
app.create_actors_relationship_with_usecase("experiments", "is", "data measurement", "usecase_018")
app.create_actors_relationship_with_usecase("experiments", "is", "policy selection", "usecase_018")
app.create_actors_relationship_with_usecase("inner loop", "has", "CDN configuration", "usecase_018")
app.create_actors_relationship_with_usecase("inner loop", "has", "QoE control", "usecase_018")
app.create_actors_relationship_with_usecase("Evolution controller", "evolve", "intent", "usecase_018")

In [ ]:
app.find_all_usecase_actors_label("usecase_018")

### FG-AN-USECASE-019

In [ ]:
app.create_actors_relationship_with_usecase("underlay", "has", "underlay specific APIs", "usecase_019")
app.create_actors_relationship_with_usecase("underlay", "has", "underlay specific configurations", "usecase_019")
app.create_actors_relationship_with_usecase("underlay", "has", "multi-domain", "usecase_019")
app.create_actors_relationship_with_usecase("underlay", "has", "network instance", "usecase_019")
app.create_actors_relationship_with_usecase("underlay", "has", "network instance", "usecase_019")
app.create_actors_relationship_with_usecase("Adaptation controller", "discovers", "underlay specific configurations", "usecase_019")

app.create_actors_relationship_with_usecase("underlay", "has", "architecture options", "usecase_019")
app.create_actors_relationship_with_usecase("underlay", "has", "deployment options", "usecase_019")
app.create_actors_relationship_with_usecase("underlay", "has", "data model", "usecase_019")
app.create_actors_relationship_with_usecase("underlay", "has", "applications", "usecase_019")
app.create_actors_relationship_with_usecase("applications", "requirement", "multi-domain", "usecase_019")
app.create_actors_relationship_with_usecase("applications", "requirement", "real time", "usecase_019")
app.create_actors_relationship_with_usecase("applications", "requirement", "non real time", "usecase_019")
app.create_actors_relationship_with_usecase("underlay orchestrator", "capability", "controller placement", "usecase_019")
app.create_actors_relationship_with_usecase("controller placement", "is", "multi-domain", "usecase_019")

In [ ]:
app.find_all_usecase_actors_label("usecase_019")

### FG-AN-USECASE-020

In [ ]:
app.create_actors_relationship_with_usecase("edge network", "is", "underlay", "usecase_020")
app.create_actors_relationship_with_usecase("edge network", "has", "Data analytics", "usecase_020")
app.create_actors_relationship_with_usecase("Data analytics", "is", "real time", "usecase_020")
app.create_actors_relationship_with_usecase("Data analytics", "is", "non real time", "usecase_020")
app.create_actors_relationship_with_usecase("verticals", "require", "Data analytics", "usecase_020")
app.create_actors_relationship_with_usecase("edge network", "has", "applications", "usecase_020")
app.create_actors_relationship_with_usecase("verticals", "require", "applications", "usecase_020")
app.create_actors_relationship_with_usecase("applications", "require", "optimization", "usecase_020")

In [ ]:
app.find_all_usecase_actors_label("usecase_020")

### FG-AN-USECASE-021

In [ ]:
app.create_actors_relationship_with_usecase("Experimentation controller", "creates", "experimentation strategies", "usecase_021")
app.create_actors_relationship_with_usecase("Experimentation controller", "executes", "experimentation strategies", "usecase_021")
app.create_actors_relationship_with_usecase("Experimentation controller", "optimizes", "experimentation strategies", "usecase_021")
app.create_actors_relationship_with_usecase("Experimentation controller", "optimizes", "experimentation strategies", "usecase_021")
app.create_actors_relationship_with_usecase("AN sandbox", "is", "underlay", "usecase_021")
app.create_actors_relationship_with_usecase("Experimentation controller", "monitors", "experiments", "usecase_021")

app.create_actors_relationship_with_usecase("Experimentation controller", "requirement", "data", "usecase_021")
app.create_actors_relationship_with_usecase("rare scenarios", "is", "experiments", "usecase_021")
app.create_actors_relationship_with_usecase("Experimentation controller", "creates", "optimization", "usecase_021")

In [ ]:
app.find_all_usecase_actors_label("usecase_021")

### FG-AN-USECASE-022

In [10]:
app.create_actors_relationship_with_usecase("underlay capabilities", "has", "domain specific aspects", "usecase_022")
app.create_actors_relationship_with_usecase("underlay capabilities", "has", "network service specific aspects", "usecase_022")
app.create_actors_relationship_with_usecase("domain specific aspects", "has", "multi-domain", "usecase_022")
app.create_actors_relationship_with_usecase("domain specific aspects", "has", "unknown", "usecase_022")
app.create_actors_relationship_with_usecase("underlay capabilities", "has", "service specific aspects", "usecase_022")
app.create_actors_relationship_with_usecase("service specific aspects", "has", "E2E", "usecase_022")
app.create_actors_relationship_with_usecase("service specific aspects", "has", "domain specific aspects", "usecase_022")
app.create_actors_relationship_with_usecase("service description", "has", "service specific aspects", "usecase_022")
app.create_actors_relationship_with_usecase("service specific aspects", "has", "sub-problems", "usecase_022")
app.create_actors_relationship_with_usecase("underlay orchestrator", "discovers", "service specific aspects", "usecase_022")

app.create_actors_relationship_with_usecase("service evolution", "solve", "sub-problems", "usecase_022")
app.create_actors_relationship_with_usecase("Evolution controller", "has", "service evolution", "usecase_022")
app.create_actors_relationship_with_usecase("service evolution", "has", "service design", "usecase_022")
app.create_actors_relationship_with_usecase("service evolution", "has", "service optimization", "usecase_022")


Node already exists: underlay capabilities in ['usecase_010_cat1', 'usecase_012_cat1', 'usecase_013_cat1', 'usecase_013_cat2']
adding label usecase_022 in addition to ['usecase_010_cat1', 'usecase_012_cat1', 'usecase_013_cat1', 'usecase_013_cat2']
Created node: domain specific aspects
Created relation: underlay capabilities - has - domain specific aspects in usecase_022
Node already exists: underlay capabilities in ['usecase_010_cat1', 'usecase_012_cat1', 'usecase_013_cat1', 'usecase_013_cat2', 'usecase_022']
ignoring ...
Created node: network service specific aspects
Created relation: underlay capabilities - has - network service specific aspects in usecase_022
Node already exists: domain specific aspects in ['usecase_022']
ignoring ...
Node already exists: multi-domain in ['usecase_013_cat2', 'usecase_014', 'usecase_016', 'usecase_019']
adding label usecase_022 in addition to ['usecase_013_cat2', 'usecase_014', 'usecase_016', 'usecase_019']
Created relation: domain specific aspects -

In [11]:
app.find_all_usecase_actors_label("usecase_022")

Found actor-1: Evolution controller
Found actor-2: underlay orchestrator
Found actor-3: underlay capabilities
Found actor-4: multi-domain
Found actor-5: domain specific aspects
Found actor-6: network service specific aspects
Found actor-7: unknown
Found actor-8: service specific aspects
Found actor-9: E2E
Found actor-10: service description
Found actor-11: sub-problems
Found actor-12: service evolution
Found actor-13: service design
Found actor-14: service optimization


### FG-AN-USECASE-023
*detailed implementation of this UC may be provided by Team Costa Rica*

In [13]:
app.create_actors_relationship_with_usecase("controllers", "has", "ML model", "usecase_023")
app.create_actors_relationship_with_usecase("controllers", "has", "SINK", "usecase_023")
app.create_actors_relationship_with_usecase("controllers", "is", "autonomous", "usecase_023")
app.create_actors_relationship_with_usecase("data", "is", "environment data", "usecase_023")
app.create_actors_relationship_with_usecase("SINK", "has", "underlay specific configurations", "usecase_023")
app.create_actors_relationship_with_usecase("underlay specific configurations", "is", "tx power", "usecase_023")
app.create_actors_relationship_with_usecase("data", "is", "multi mode", "usecase_023")
app.create_actors_relationship_with_usecase("data", "is", "multi-mode", "usecase_023")
app.create_actors_relationship_with_usecase("multi-mode", "is", "image", "usecase_023")
app.create_actors_relationship_with_usecase("controllers", "has", "SRC", "usecase_023")
app.create_actors_relationship_with_usecase("controllers", "discover", "SRC", "usecase_023")
app.create_actors_relationship_with_usecase("root cause analysis", "is", "ML model", "usecase_023")
app.create_actors_relationship_with_usecase("root cause analysis", "has", "failure", "usecase_023")
app.create_actors_relationship_with_usecase("failure", "has", "software failure", "usecase_023")
app.create_actors_relationship_with_usecase("failure", "has", "hardware failure", "usecase_023")
app.create_actors_relationship_with_usecase("failure", "has", "middleware failure", "usecase_023")
app.create_actors_relationship_with_usecase("underlay", "has", "version", "usecase_023")
app.create_actors_relationship_with_usecase("controllers", "has", "version", "usecase_023")

app.create_actors_relationship_with_usecase("sub-problems", "has", "runtime", "usecase_023")


Node already exists: controllers in ['usecase_001', 'usecase_004', 'usecase_006_cat1', 'usecase_007_cat1', 'usecase_007_cat2', 'usecase_008_cat1', 'usecase_009_cat1', 'usecase_009_cat2', 'usecase_010_cat1', 'usecase_010_cat2', 'usecase_011_cat1', 'usecase_011_cat2', 'usecase_013_cat2', 'usecase_016', 'usecase_017']
adding label usecase_023 in addition to ['usecase_001', 'usecase_004', 'usecase_006_cat1', 'usecase_007_cat1', 'usecase_007_cat2', 'usecase_008_cat1', 'usecase_009_cat1', 'usecase_009_cat2', 'usecase_010_cat1', 'usecase_010_cat2', 'usecase_011_cat1', 'usecase_011_cat2', 'usecase_013_cat2', 'usecase_016', 'usecase_017']
Node already exists: ML model in ['usecase_001', 'usecase_006_cat2', 'usecase_007_cat1', 'usecase_015', 'usecase_017', 'usecase_018']
adding label usecase_023 in addition to ['usecase_001', 'usecase_006_cat2', 'usecase_007_cat1', 'usecase_015', 'usecase_017', 'usecase_018']
Created relation: controllers - has - ML model in usecase_023
Node already exists: cont

In [14]:
app.find_all_usecase_actors_label("usecase_023")

Found actor-1: controllers
Found actor-2: ML model
Found actor-3: underlay
Found actor-4: data
Found actor-5: underlay specific configurations
Found actor-6: SINK
Found actor-7: autonomous
Found actor-8: environment data
Found actor-9: tx power
Found actor-10: multi mode
Found actor-11: multi-mode
Found actor-12: image
Found actor-13: SRC
Found actor-14: root cause analysis
Found actor-15: failure
Found actor-16: software failure
Found actor-17: hardware failure
Found actor-18: middleware failure
Found actor-19: version


### FG-AN-USECASE-024

In [16]:
app.create_actors_relationship_with_usecase("underlay", "is", "adaptive hardware", "usecase_024")
app.create_actors_relationship_with_usecase("design", "has", "design tradeoff", "usecase_024")
app.create_actors_relationship_with_usecase("controller", "has", "design", "usecase_024")
app.create_actors_relationship_with_usecase("Evolution controller", "optimizes", "design", "usecase_024")

app.create_actors_relationship_with_usecase("edge network", "has", "ML model", "usecase_024")
app.create_actors_relationship_with_usecase("edge network", "requirement", "low latency", "usecase_024")
app.create_actors_relationship_with_usecase("edge network", "requirement", "low power consumption", "usecase_024")
app.create_actors_relationship_with_usecase("edge network", "requirement", "low memory", "usecase_024")

app.create_actors_relationship_with_usecase("FPGA", "is", "adaptive hardware", "usecase_024")
app.create_actors_relationship_with_usecase("adaptive System-on-Chip", "is", "adaptive hardware", "usecase_024")
app.create_actors_relationship_with_usecase("adaptive hardware", "capability", "customization", "usecase_024")
app.create_actors_relationship_with_usecase("design tradeoff", "is", "concurrency", "usecase_024")
app.create_actors_relationship_with_usecase("design tradeoff", "is", "buffering between layers", "usecase_024")
app.create_actors_relationship_with_usecase("design tradeoff", "is", "offloading of layers", "usecase_024")
app.create_actors_relationship_with_usecase("design tradeoff", "is", "precision", "usecase_024")
app.create_actors_relationship_with_usecase("design tradeoff", "is", "performance", "usecase_024")
app.create_actors_relationship_with_usecase("design tradeoff", "is", "energy efficiency", "usecase_024")
app.create_actors_relationship_with_usecase("ML model", "has", "model architecture", "usecase_024")
app.create_actors_relationship_with_usecase("ML model", "has", "intermediate representation ", "usecase_024")
app.create_actors_relationship_with_usecase("ML model", "requirement", "optimization", "usecase_024")
app.create_actors_relationship_with_usecase("ML model", "requirement", "hardware adaptation", "usecase_024")
app.create_actors_relationship_with_usecase("ML model", "requirement", "training", "usecase_024")
app.create_actors_relationship_with_usecase("ML model", "requirement", "evaluation", "usecase_024")

Failed to read from defunct connection ResolvedIPv4Address(('34.126.114.186', 7687)) (IPv4Address(('34.126.114.186', 7687)))
Unable to retrieve routing information
Transaction failed and will be retried in 1.041727741733592s (Unable to retrieve routing information)


Node already exists: underlay in ['usecase_005', 'usecase_010_cat1', 'usecase_011_cat1', 'usecase_014', 'usecase_019', 'usecase_020', 'usecase_021', 'usecase_023']
adding label usecase_024 in addition to ['usecase_005', 'usecase_010_cat1', 'usecase_011_cat1', 'usecase_014', 'usecase_019', 'usecase_020', 'usecase_021', 'usecase_023']
Created node: adaptive hardware
Created relation: underlay - is - adaptive hardware in usecase_024
Created node: design
Created node: design tradeoff
Created relation: design - has - design tradeoff in usecase_024
Node already exists: controller in ['usecase_018']
adding label usecase_024 in addition to ['usecase_018']
Node already exists: design in ['usecase_024']
ignoring ...
Created relation: controller - has - design in usecase_024
Node already exists: Evolution controller in ['usecase_006_cat1', 'usecase_008_cat1', 'usecase_009_cat1', 'usecase_010_cat1', 'usecase_011_cat1', 'usecase_012_cat1', 'usecase_013_cat1', 'usecase_018', 'usecase_022']
adding la

In [17]:
app.find_all_usecase_actors_label("usecase_024")

Found actor-1: ML model
Found actor-2: underlay
Found actor-3: Evolution controller
Found actor-4: optimization
Found actor-5: evaluation
Found actor-6: controller
Found actor-7: edge network
Found actor-8: adaptive hardware
Found actor-9: design
Found actor-10: design tradeoff
Found actor-11: low latency
Found actor-12: low power consumption
Found actor-13: low memory
Found actor-14: FPGA
Found actor-15: adaptive System-on-Chip
Found actor-16: customization
Found actor-17: concurrency
Found actor-18: buffering between layers
Found actor-19: offloading of layers
Found actor-20: precision
Found actor-21: performance
Found actor-22: energy efficiency
Found actor-23: model architecture
Found actor-24: intermediate representation 
Found actor-25: hardware adaptation
Found actor-26: training


### FG-AN-USECASE-025

In [ ]:
app.create_actors_relationship_with_usecase("evolution algorithms", "requirement", "ML model", "usecase_025")
app.create_actors_relationship_with_usecase("ML models", "requirement", "transfer learning", "usecase_025")
app.create_actors_relationship_with_usecase("Knowledge Base", "capability", "transfer learning", "usecase_025")
app.create_actors_relationship_with_usecase("AN orchestrator", "manages", "Knowledge Base", "usecase_025")

app.create_actors_relationship_with_usecase("environment data", "is", "data", "usecase_025")
app.create_actors_relationship_with_usecase("environment data", "is", "signal strength", "usecase_025")
app.create_actors_relationship_with_usecase("environment data", "is", "mobility data", "usecase_025")
app.create_actors_relationship_with_usecase("user model", "is", "data", "usecase_025")
app.create_actors_relationship_with_usecase("Adaptation controller", "requirement", "adaptation algorithms", "usecase_025")
app.create_actors_relationship_with_usecase("adaptation algorithms", "requirement", "validation", "usecase_025")
app.create_actors_relationship_with_usecase("controllers", "requirement", "adaptation algorithms", "usecase_025")
app.create_actors_relationship_with_usecase("underlay specific configurations", "is", "coverage parameters", "usecase_025")
app.create_actors_relationship_with_usecase("underlay specific configurations", "is", "beam parameters", "usecase_025")
app.create_actors_relationship_with_usecase("ML model", "requirement", "retraining", "usecase_025")
app.create_actors_relationship_with_usecase("simulators", "requirement", "controllers", "usecase_025")




In [18]:
app.find_all_usecase_actors_label("usecase_025")

## Final Step: close the database connection

In [ ]:
app.close()